# HST's Tranisting Exoplanet Noise Simulator  (TExoNS)

This file demonstrates how to use TExoNS to predict the:
    1. Transmission/emission spectrum S/N ratio
    2. Observation start window
for any system observed with WFC3/IR.

#### Background information
    Pandeia: ETC for JWST
    PandExo: Exoplanet noise simulator for JWST

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os, sys
sys.path.append('..')
import pandexo.engine.justdoit as jdi

## Edit Inputs

### Load in a blank exoplanet dictionary 

In [2]:
exo_dict = jdi.load_exo_dict()

#### Edit stellar and planet inputs

In [3]:
#WASP-43
exo_dict['star']['mag']      = 9.397                # H magnitude of the system
#WASP-43b
exo_dict['planet']['type']    = 'user'               # user specified inputs
exo_dict['planet']['exopath'] = 'WASP43b-Eclipse_Spectrum.txt' # filename for model spectrum
exo_dict['planet']['w_unit']  = 'um'                 # wavelength unit
exo_dict['planet']['f_unit']  = 'fp/f*'              # flux ratio unit (can also put "rp^2/r*^2")
exo_dict['planet']['depth']   = 4.0e-3               # flux ratio
exo_dict['planet']['i']       = 82.6                 # Orbital inclination in degrees
exo_dict['planet']['ars']     = 5.13                 # Semi-major axis / stellar radius
exo_dict['planet']['period']  = 0.8135               # Orbital period in days   
exo_dict['planet']['transit_duration']= 4170/60/60/24#(optional if given above info) transit duration in days
exo_dict['planet']['w']       = 90                   #(optional) longitude of periastron. Default is 90
exo_dict['planet']['ecc']     = 0                    #(optional) eccentricity. Default is 0 

### Step 2) Load in instrument dictionary
 
- WFC3 G141
- WFC3 G102

In [4]:
inst_dict = jdi.load_mode_dict('WFC3 G141')

#### Edit HST/WFC3 detector and observation inputs

In [5]:
inst_dict['configuration']['detector']['subarray']     = 'GRISM256'   # GRISM256 or GRISM512
inst_dict['configuration']['detector']['nsamp']        = 10           # WFC3 N_SAMP, 1..15
inst_dict['configuration']['detector']['samp_seq']     = 'SPARS5'     # WFC3 SAMP_SEQ, SPARS5 or SPARS10
exo_dict['observation']['norbits']                     = 4            # Number of HST orbits
exo_dict['observation']['noccultations']               = 5            # Number of transits/eclipses
exo_dict['observation']['nchan']                       = 15           # Number of spectrophotometric channels
exo_dict['observation']['scanDirection']               = 'Forward'    # Spatial scan direction, Forward or Round Trip
exo_dict['observation']['schedulability']              = 30           # 30 for small/medium program, 100 for large program
exo_dict['observation']['windowSize']                  = 20           # (optional) Observation start window size in minutes. Default is 20 minutes.

## Run PandExo

`    jdi.run_pandexo(exo, inst, param_space = 0, param_range = 0,save_file = True,
                            output_path=os.getcwd(), output_file = '')`

See wiki Attributes for more thorough explanation fo inputs


In [6]:
foo = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')

Running Single Case w/ User Instrument Dict
****WARNING: Observing plan may incur mid-orbit buffer dumps.  Check with APT.


In [7]:
inst_dict['configuration']['detector']['nsamp'] = None
inst_dict['configuration']['detector']['samp_seq'] = None
bar = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')

Running Single Case w/ User Instrument Dict


In [8]:
exo_dict['observation']['scanDirection'] = 'Round Trip'
hst = jdi.run_pandexo(exo_dict, inst_dict, output_file='wasp43b.p')

Running Single Case w/ User Instrument Dict


## Plot Results


#### Plot simulated spectrum using specified file

In [13]:
import pandexo.engine.justplotit as jpi 

datawave, dataspec, dataerror, modelwave, modelspec = jpi.hst_spec(foo)

[ 1.1425  1.1775  1.2125  1.2475  1.2825  1.3175  1.3525  1.3875  1.4225
  1.4575  1.4925  1.5275  1.5625  1.5975  1.6325] [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan] [ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan] [ 10.           9.98003992   9.96015936 ...,   1.05307498   1.05285323
   1.05263158] [ 0.00776939  0.00770963  0.00765077 ...,  0.00029181  0.00029181
  0.00029181]


ValueError: Out of range float values are not JSON compliant: nan

#### Compare simulation to published results

In [ ]:
print("Simulated uncertainty: %0.1f ppm" % (deptherr*1e6))
print("Mean published uncertainty: %0.1f +/- %0.1f ppm" % (np.mean(model[2][:15])*1e6,np.std(model[2][:15])*1e6))
print("Results are consistent at %0.1f sigma." % ((deptherr-np.mean(model[2][:15]))/np.std(model[2][:15])))

## Compute earliest and latest possible start times for given start window size

In [ ]:
numorbits = exo_dict['observation']['norbits']
depth     = exo_dict['planet']['depth']
inc       = exo_dict['planet']['i']
aRs       = exo_dict['planet']['ars']
period    = exo_dict['planet']['period']
#duration = exo_dict['observation']['trdur']/86400. # Duration in days
tunc      = 10                                      # +/- start time uncertainty in minutes
minphase, maxphase = hst.calc_StartWindow(rms, ptsOrbit, numorbits, depth, inc, aRs, period, tunc)

In [ ]:
print("Start observations between orbital phases: %0.4f - %0.4f" % (minphase, maxphase))

### ToDo List:
    
    Add functionality for STIS G430 and STIS G750 grisms
    Compute wavelength-dependent uncertainties for STIS
    Develop web interface (using PandExo) for end users
    Add additional calculation methods (currently scaling flux/variance from actual observations)